In [1]:
#load path
import os
import sys
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus
from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob


In [2]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
reduxpath='./data/Redux/'
combinepath='./data/Redux/Combined/'
gain=0.25 #e-/ADU at G = 125

In [3]:
def GAIA_query(center_ra, center_dec, radius, magmin=12, magmax=13, writedir='./calstars/'):
   from astroquery.gaia import GaiaClass
   from astropy.coordinates import SkyCoord
   import astropy.units as u


   coord = SkyCoord(ra=center_ra, dec=center_dec, unit=(u.deg, u.deg))
   gaiaobj = GaiaClass()
   gaiaobj.ROW_LIMIT = 50000  # High limit on number of rows returned
   job = gaiaobj.cone_search(coord, radius=radius * u.deg, verbose=False)
   results = job.get_results()

   selected = (results['has_xp_sampled']) & (results['phot_g_mean_mag'] < magmax) & (results['phot_g_mean_mag'] > magmin)
   starID = results['source_id'][selected]
   starRA = results['ra'][selected]
   starDEC = results['dec'][selected]

   print('Found {} stars'.format(len(starID)))
   ## Download the spectra:
   retrieval_type = 'XP_SAMPLED'
   datalink = gaiaobj.load_data(ids=starID,
                          data_release='Gaia DR3',
                          retrieval_type=retrieval_type,
                          data_structure='INDIVIDUAL',
                          format='votable',
                          verbose=False,
                          overwrite_output_file='True')

   dl_keys  = [inp for inp in datalink.keys()]

   strwave = []
   strflux = []

   #flux is in w/m2/nm, convert to erg/s/cm2/A
   conv_factor = 1e7 * 1e-4 * 1e-1 #1 W = 1e7 erg/s, 1 m2 = 1e4 cm2, 1 nm = 10 A

   for ind, strid in enumerate(starID):
         for dl_key in dl_keys:
              if str(strid) in dl_key:
                wave = datalink[dl_key][0].to_table().as_array()['wavelength']
                wave = np.ma.getdata(wave)*10
                flux = datalink[dl_key][0].to_table().as_array()['flux']
                flux = np.ma.getdata(flux)*conv_factor
                strwave.append(wave)
                strflux.append(flux)
         #Save in fits file
         hdu1 = fits.PrimaryHDU(flux)
         hdu1.header['SRCID'] = starID[ind]
         hdu1.header['RA'] = starRA[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['FUNIT'] = 'erg/s/cm2/A'
         hdu1.header['CRVAL1'] = wave[0]
         hdu1.header['CD1_1'] = wave[1]-wave[0]
         hdu2 = fits.ImageHDU(wave)
         hdulist = fits.HDUList([hdu1, hdu2])
         hdulist.writeto(writedir+'GAIA3_{}.fits'.format(starID[ind]))
    
   return starID, starRA, starDEC, np.array(strwave), np.array(strflux)

In [4]:
def make_star_library(header, img, caldir, force_query=False):

    #Find center of image in RA and DEC
    wcs = WCS(header)
    ny , nx = img.shape
    center_ra, center_dec = wcs.wcs_pix2world(nx/2, ny/2, 0)

    objname = header['OBJECT'].replace(' ','_')
    caltargdir = caldir+objname+'/'
    if not os.path.exists(caltargdir):
       os.makedirs(caltargdir)

    fcalstr = glob.glob(caltargdir+'GAIA3_*.fits')
    if len(fcalstr) == 0:

      print('Running GAIA query....')  
      #Query GAIA for stars in the field
      radius = 0.3 #degrees
      strID, strRA, strDEC, specwave, specflux = GAIA_query(center_ra, center_dec, radius, \
                                                 magmin=12, magmax=13, writedir=caltargdir)
    else:
      print('Reading GAIA spectra from: {}'.format(caltargdir))
      strID = []
      strRA = []
      strDEC = []
      specwave = []
      specflux = []

      for fcal in fcalstr:
          hdu = fits.open(fcal)
          strID.append(hdu[0].header['SRCID'])
          strRA.append(hdu[0].header['RA'])
          strDEC.append(hdu[0].header['DEC'])
          specwave.append(hdu[1].data)
          specflux.append(hdu[0].data)
          hdu.close()

      specwave = np.array(specwave)
      specflux = np.array(specflux) 
      
    #For each star, find its pixel position in the image
    star_positions = []
    for ra, dec in zip(strRA, strDEC):
        x, y = wcs.wcs_world2pix(ra, dec, 0)
        star_positions.append((x, y))
    star_positions = np.array(star_positions)

    #keep only stars within the image boundaries
    okstars =  (0 < star_positions[:,0]) & (star_positions[:,0] < nx) \
               & (star_positions[:,1] < ny) & (0 < star_positions[:,1])        
    
    star_positions = star_positions[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]

    return star_positions, specwave, specflux


In [5]:
def flux_cal_gaia(img, header, filter, caldir='./calstars/'):

    star_positions, specwave, specflux = make_star_library(header, img, caldir)
    if len(star_positions)==0:
       star_positions, specwave, specflux = make_star_library(header, img, caldir, force_query=True) 
       if len(star_positions)==0:
           return 0, 0
         
    #Perform aperture photometry on each star with background subtraction
    apertures = CircularAperture(star_positions, r=7)
    annuli = CircularAnnulus(star_positions, r_in=7, r_out=10)
    apers = [apertures, annuli]
    phot_table = aperture_photometry(img, apers)
    bkg_mean = phot_table['aperture_sum_1'] / annuli.area
    bkg_sum = bkg_mean * apertures.area
    inst_flux = phot_table['aperture_sum_0'] - bkg_sum

    #Keep only stars with positive fluxes
    okstars = (inst_flux>0) & (np.isfinite(inst_flux))
    inst_flux = inst_flux[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]    

    #Read appropriate filter file
    filterdict = {'R':'r', 'G':'g', 'I':'i', 'SII':'SII', 'OIII':'OIII', 'Ha':'Ha', 'Hb':'Hb'}
    filterfile = './filters/Tobi_'+filterdict[filter]+'.txt'
    filterdata = np.loadtxt(filterfile)

    #convolve each spectrum with the filter
    filter_wave = filterdata[:,1]
    filter_trans = filterdata[:,2]  
    calfactors = []
    
    for i in range(okstars.sum()):
        wave = specwave[i]
        flux = specflux[i]
        #interpolate filter to spectrum wavelength
        filter_interp = np.interp(wave, filter_wave, filter_trans, left=0, right=0)
        #compute the integral of flux * filter
        num = np.trapz(flux * filter_interp, wave)
        denom = np.trapz(filter_interp, wave)
        flux_cal = num / denom  # erg/s/cm2/A
        #compute the calibration factor
        calfactor = flux_cal / inst_flux[i]  # erg/s/cm2/A per e-/s
        calfactors.append(calfactor)   

    #return median calibration factor   
    median_calfactor = np.median(calfactors)
    std_calfactor = np.std(calfactors)
    
    print('Calibration factor for image in filter {}: {:.2e} +/- {:.2e}'.format(filter, median_calfactor, std_calfactor))
    
    return median_calfactor, std_calfactor 




In [6]:
#now reduce science images
#in every directory of rawpath, find if there is a LIGHT folder and reduce the images inside
diffrun=False
for root, dirs, files in os.walk(rawpath):
    if 'LIGHT' in dirs:
        light_dir = os.path.join(root, 'LIGHT')
        sciencelist=glob.glob(light_dir+'/*sci*.fits')
        masterbias=fits.open(calibpath+'/MasterBias.fits')[0].data
        for img in sciencelist:
          if diffrun and os.path.isfile(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1]):
              print('Skipping file {}'.format(img))
              continue
          else:
           hdu=fits.open(img)
           data=hdu[0].data
           header=hdu[0].header
           filter=header['FILTER']
           exptime=header['EXPTIME']
           normflat=fits.open(calibpath+'MasterFlat_'+filter+'.fits')[0].data
           reduced=data-masterbias
           reduced=reduced/normflat/exptime*gain

           #Crop 300 pixels from each edge to avoid vignetting, preserve WCS
           reduced = reduced[300:-300,300:-300]
           header['CRPIX1'] -= 300
           header['CRPIX2'] -= 300

           #Now run fluxcal
           calfac, calstd = flux_cal_gaia(reduced, header, filter, caldir='./calstars/')
           if calfac !=0:
              reduced *= calfac /1e-17

           header['FCALFAC'] = calfac 
           header['FCALSTD'] = calstd
           header['FUNIT'] = '1e-17 erg/s/cm2/A'   

           fits.writeto(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1],reduced,header,overwrite=True)

Set MJD-AVG to 60950.833946 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/NGC6946/
Calibration factor for image in filter I: 1.07e-16 +/- 7.23e-18


Set MJD-AVG to 60950.844570 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/NGC6946/
Calibration factor for image in filter G: 6.52e-17 +/- 3.50e-18


Set MJD-AVG to 60950.826507 from DATE-AVG'. [astropy.wcs.wcs]


...

Reading GAIA spectra from: ./calstars/BD25/
Reading GAIA spectra from: ./calstars/BD25/


Set MJD-AVG to 60946.810205 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/BD28/
Calibration factor for image in filter G: 6.60e-17 +/- 2.51e-17


Set MJD-AVG to 60946.848429 from DATE-AVG'. [astropy.wcs.wcs]


Reading GAIA spectra from: ./calstars/BD25/
Calibration factor for image in filter SII: 2.35e-15 +/- 1.02e-16


In [ ]:
'''
#astrometrically solve the images
ast = AstrometryNet()
ast.api_key = 'ywslonowruohigzw'
reduximages = glob.glob(reduxpath+'/*_reduced.fits')

for fileout in reduximages:
    wcshdu = fits.open(fileout)
    try:
        haswcs = wcshdu[0].header['WCSAXES']
        print('Do not overwrite WCS for', fileout)
    except KeyError:
        print('Process WCS for', fileout)
        wcs_header = ast.solve_from_image(fileout, force_image_upload=True, publicly_visible='n',solve_timeout=3600)
        if wcs_header:
            wcshdu = fits.open(fileout)
            wcshdu[0].header += wcs_header
            wcshdu.writeto(fileout, overwrite=True)
            print("Writing WCS to", fileout)
        else:
           print('FAIL, WCS not modified')
'''

In [ ]:
#for each image, run sep, mask sources and fit sky background
import sep
import numpy as np

reduximages = glob.glob(reduxpath+'/*_reduced.fits')
diffrun=True

for fileout in reduximages:
   if diffrun and os.path.isfile(fileout.replace('_reduced.fits','_sky_sub.fits')):
    print('Skipping file {}'.format(fileout))
    continue
   else:
    hdu = fits.open(fileout)
    data = (hdu[0].data).astype(np.float32)
    bkg = sep.Background(data)
    data_sub = data - bkg
    sources, segmap = sep.extract(data_sub, 2.0, minarea=15, err=bkg.globalrms, segmentation_map=True)
    segmap[segmap>0] = 1
    sky_median = np.median(data[segmap==0])
    maskravel = (segmap==0).ravel()
    #fit a 2D polynomial to the sky background using astropy
    y, x = np.mgrid[:data.shape[0], :data.shape[1]]
    A = np.c_[x.ravel()**2, y.ravel()**2, x.ravel()*y.ravel(), x.ravel(), y.ravel(), np.ones(data.shape).ravel()]
    C, _, _, _ = np.linalg.lstsq(A[maskravel], data.ravel()[maskravel], rcond=None)
    sky_fit = (C[0]*x**2 + C[1]*y**2 + C[2]*x*y + C[3]*x + C[4]*y + C[5]).reshape(data.shape)
    data_sky_sub = data - sky_fit

    hdu[0].data = data_sky_sub
    hdu.writeto(fileout.replace('_reduced.fits','_sky_sub.fits'), overwrite=True)


Coadding

In [ ]:
import glob
from astropy.io import fits
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np

# List of filters to process
filters = ['R','G','I','SII','OIII','Ha','Hb']
targname = 'NGC7331'

for filter in filters:
    # Find all sky-subtracted images for this filter
    images = glob.glob(reduxpath + f'/*{targname}_{filter}*sky_sub.fits')
    if len(images) == 0:
        print('No images found')
        continue
    else:
        print(f'Processing {len(images)} images for filter {filter}')

    # Use the first image as reference
    ref_hdu = fits.open(images[0])
    ref_data = ref_hdu[0].data
    ref_header = ref_hdu[0].header
    ref_wcs = WCS(ref_header)
    stack = []

    for img in images:
        hdu = fits.open(img)
        data = hdu[0].data
        header = hdu[0].header
        wcs = WCS(header)
        # Reproject to reference WCS
        reproj_data, _ = reproject_interp((data, wcs), ref_wcs, shape_out=ref_data.shape)
        stack.append(reproj_data)

    # Combine (median)
    stack = np.array(stack)
    combined = np.median(stack, axis=0)

    # Save combined image
    fits.writeto(combinepath + f'/Combined_{targname}_{filter}.fits', combined, ref_header, overwrite=True)